# TME 3.1 : Greedy Profile Motif Search 


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm Greedy Profile Motif Search.


1\. Nous allons réutiliser les fonctions du precedent pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [189]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [24]:
from math import floor

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range (n):
        tmp=random.choice(nuc)
        sequence =sequence+tmp
        
    if upper== False:
        sequence= sequence.lower()
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    tmp=''
    randind=0
    up=upper
    
    listIndDéjaMODIF=[]
    
    for pos in range(nbpos): #On modifie autant de position possible que la taille du motif le permet 
        if pos < len(motifM)-1:
            #Tirer un caractere aléatoire tmp parmi atgc et l'insérer à la posiion randind choisie aléatoirement 
            tmp = generateRandomSequence(1, up)
            randind= random.randint(0, len(motif)-1)
            
            #Pour ne pas modifier à 2 reprises à un meme indice
            if randind in listIndDéjaMODIF: #On doit tirer un nouvel indice aléatoirevement
                
                indConnue = True
                while indConnue:
                    randind= random.randint(0, len(motif)-1)
                    if randind not in listIndDéjaMODIF:
                        indConnue = False
            
            #Modification à la position choisit aléatoirement
            motifM[randind]=tmp
            listIndDéjaMODIF.append(randind)
        
    return "".join(motifM)


#tester la fonction modifierMotif
print (modifierMotif("acg", 2,True))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    sequences = []
    motifRand=generateRandomSequence(k,False)
    print('Motif à implanté : ',motifRand)
    motifTMP=''
    seqTPM=''
    tailleSeqSanMotifImplanté = n-k
    positionRand=random.randint(0,tailleSeqSanMotifImplanté)
    
    #Détermine nb de séquence pour lesquelles le motif implanté ne doit pas varier, dans f des cas le motif ne doit varier
    SeqAvecVarDeMotif= floor(f*t) #Partie entiére inférieure 
    Compteur=0
    
    for sep in range(t): #Création des séquences avec le motif générer qui varie de v base et pour une fréquence f
        seqTPM=generateRandomSequence(tailleSeqSanMotifImplanté,True) #Sequence en majuscule
        
        #Choix du motif
        if Compteur == SeqAvecVarDeMotif: 
            #Stop l'insertion de Motif avec variation
            motifTMP=motifRand
        else: 
            motifTMP= modifierMotif (motifRand,v,False)        #Motif en minuscule
            Compteur = Compteur+1
        
        #Insertion du motif dans la seq et ajout à la liste résultante cad la matrice
        seqTPM= insertMotif(seqTPM, motifTMP, positionRand)
        sequences.append(seqTPM) #list(seqTPM)
    
        #Initialisation d'une position pour insérer motif de la prochaine séquence 
        positionRand=random.randint(0,tailleSeqSanMotifImplanté) 
        
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [("".join(s)).upper() for s in adn]
print (adn)

GCg
tcaca
['TTGACccaca', 'AGACtcacaC', 'TtcacaCAAG']
['TTGACCCACA', 'AGACTCACAC', 'TTCACACAAG']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [29]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    """
    Génère un vecteur de position aléatoires
    entrée sequences: matrice de dimension txn contenant les sequences
    entrée k: taille du motif
    sortie s: vecteur de position aléatoires, une position par séquence
    REMARQUE les positions doivent être inférieur à n-k, ou n est la taille de sequence
    """
    s = []
    
    for sq in (sequences):
        if len(sq)-k > 0:
            randind= random.randint(0, len(sq)-k)
            s.append(randind)
    return s

s = generateRandomS(adn, k)
print (s)

[4, 5, 4]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). 


In [36]:
#extract sequences from seqs
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    motifs = []    
    
    for ind in range(len(seqs)) :
        sq=seqs[ind]
        deb= s[ind]
        
        motif= sq[deb:deb+k] #Extraction du motif
        motifs.append(motif) #Ajout du motif à la liste résultante
   
    return motifs
        
motifs = extractSeqs(s, adn, k)
print (motifs)

#Construire un profile = matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    #nuc = ('A', 'C', 'G', 'T')
    q = len(nuc)
    PWM = np.zeros((q, k))
    
    for mot in motifs :
        for i in range (k): #i=indice colonne
            indLigne = nuc.index(mot[i]) #indiceLigne
            PWM[indLigne,i]=PWM[indLigne,i]+1
            
    return PWM

MF = profile(motifs, k, nuc)
print(MF)                       

['CCCAC', 'CACAC', 'CACAA']
[[0. 2. 0. 3. 1.]
 [3. 1. 3. 0. 2.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [41]:
def generatePWM(MF):
    """
    Transforme la matrice de fréquence en PWM
    entrée MF: matrice de fréquence
    sortie PWM: matrice de probabilité (poids positions)
    """
    
    PWM = MF
    nbLigne = len(MF)
    nbColonne = len(MF[0])
    
    #Calcule du nombre fréquence par colonne (En réalité le nombre de fréquence = nb de ligne dans la matrice)
    freq = []
    for c in range(nbColonne):
        tmp=0
        for l in range(nbLigne):
            tmp=tmp + MF[l,c]
        freq.append(tmp)
   
    #Pour chaque ligne on boucle sur les colonne et divison chaque cellule par fréquence de sa colonne
    for l in range(nbLigne): 
        for c in range(nbColonne):
            PWM[l,c]=MF[l,c]/freq[c]
    
    return PWM

PWM = generatePWM(MF)
print(PWM)

[[0.         0.66666667 0.         1.         0.33333333]
 [1.         0.33333333 1.         0.         0.66666667]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [60]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    prob = 1
    nuc = ('A', 'C', 'G', 'T')
    
    for col in range(len(motif)):
        indLigne = nuc.index(motif[col]) #indiceLigne
        #print(PWM[indLigne,col])
        prob= prob* PWM[indLigne,col]

    return prob

prob = probability(PWM, "AAACA")
print (prob)

0.0


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [77]:
def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position la plus probable 
    """
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence
    
    for i in range(len(sequence)-k+1): 
        tmp= probability(PWM, sequence[i:i+k]) #Fenètre glissante pour calculer proba du motif considérer dans la sequence en fct de PWM
        #print("indice :",i,"proba motif: ",tmp)
        if tmp>maxProb: #Màj en cas d'un motif plus probable déterminer
            maxProb=tmp
            s=i  
        
    return s
s = pMostProbkmer(PWM, k, "GCCGGATACT") #Du G et T dans chaque motif de la fentre glissante, conséquence le motif à la proba la plus importante reste celui de debut avec indice 0 pr départ
print (s, "\n")

#s = pMostProbkmer(PWM, k, "GCCGGCCCAA")
#print ("Test cette fois en mettant le dernier motid rencontrer à un motif plus probable (Effectivement c'eest le bon indice qui est renvoyer) : ",s)

0 



7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [78]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """
    s = []
    
    for sq in sequences:
        tmp=pMostProbkmer(PWM, k, sq)
        s.append(tmp)

    return s

ns = getNewS(PWM, k, adn)
print (ns)

[4, 5, 2]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [119]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    
    #Comparaison des matrices de fréquence
    difference=0
    nbligne=min(len(P1),len(P2))
    nbcol=min(len(P1[0]),len(P2[0]))
    
    for l in range(nbligne):
        for c in range(nbcol):
            if P1[l,c] != P2[l,c]:
                difference=difference+1
                break
        if difference >= 1:
            break
    
    if difference == 0:
        return False
    
    return True

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    nbLigne = len(MF)
    nbColonne = len(MF[0])
    
    for c in range(nbColonne):
        MAX=0
        for l in range(nbLigne):
            #print(MF[l,c])
            if MF[l,c]>MAX:
                MAX=MF[l,c]
        #print("Sortie")
        sc= sc + MAX
        #print("Valeur sc =", sc)

    return sc

ns = getNewS(PWM, k, adn)
print (s, ns)

#MF = profile(motifs, k, nuc)
sc = getScore(MF, k)
print (sc)

#Test fonction changeProfile
MFtest1=np.array([[3, 2, 0, 3, 1],[3, 1, 3, 0, 2],[0, 0, 0, 0, 0],[0, 0, 0, 0, 0]])
MFtest2=np.array([[3, 2, 0, 3, 1],[3, 1, 3, 0, 2],[0, 0, 0, 0, 0],[0, 0, 0, 0, 9]])
#print(MFtest1)
#print(MFtest2)
print(changeProfile(MFtest1, MFtest2))

0 [4, 5, 2]
13.0
True


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 
BONUS: Pour ameliorer la performance vous pouvez abandonner les positions qui génèrent des motifs peu complexes.

In [123]:
def GreedyProfileMotifSearch(sequences, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
    entrée sequences: matrice de dimension txn contenant les séquences 
    entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    #NOUS ENVISAGEONS D'OPTIMISER LA FONCTION POUR IGNORER LES MOTIFS PEU COMPLEX, APRÉS LE DÉPOT
    
    nuc = ('A', 'C', 'G', 'T')
    bestScore = 0
    
    s = generateRandomS(sequences, k)
    motifs=extractSeqs(s, sequences, k) #Liste des motifs plus probable
    MF=profile(motifs, k, nuc) #Détermine le Profile/MF
    PWM=generatePWM(MF) #Détermine la matrice de proba associée
    score =getScore(MF,k) #Score associé à la MF courante déterminer par les motifs les plus probables(P-most probable k-mers) jusqu'à présent obtenue
    
    #Variables temporaires pour les itérations
    tmpS=getNewS(PWM, k, sequences)
    tmpMotifs=extractSeqs(tmpS, sequences, k)
    tmpMF=profile(tmpMotifs, k, nuc)
    
    while score>bestScore:
        bestScore= score
        
        if (changeProfile(MF, tmpMF)): #Profil sont différents
            #Cas bestscore < score : on continue l'algo
            if getScore(tmpMF,k) > bestScore:
                
                #Màj des paramètres liés au motif les plus probables de nos séquances
                bestScore=getScore(tmpMF,k)
                motifs=tmpMotifs
                MF=tmpMF
                PWM=generatePWM(MF) #Génère une new matrice de proba pour déterminer une nouvelle matrice de position
    
                #Anticipation de la prochaine itération en mettant à jour les variables temporaires
                tmpS=getNewS(PWM, k, sequences)
                tmpMotifs=extractSeqs(tmpS, sequences, k)
                tmpMF=profile(tmpMotifs, k, nuc)
                score=getScore(tmpMF,k)
 
            #cas score < bestscore
            else :
                break
        
        else: #Profil ont pas change on stop l'algo entre 2 itérations
            break
              
    return s, bestScore

print (adn)
s, bestScore = GreedyProfileMotifSearch(adn, k)   
motifs = extractSeqs(s, adn, k)
print (motifs)
        

['TTGACCCACA', 'AGACTCACAC', 'TTCACACAAG']
['GACCC', 'TCACA', 'CACAC']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse:

'tcaca' est notre motif implanté dans 50% de nos séquences sans variation et nous le retrouvons effectivement ^^

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [136]:
def GreedyProfileMotifSearchIte(sequences, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
	  entrée séquences: matrice de dimension txn contenant les séquences
	  entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= sequence consensus du motif
    """
       
    #NOUS ENVISAGEONS D'OPTIMISER LA FONCTION POUR IGNORER LES MOTIFS PEU COMPLEX, APRÉS LE DÉPOT
    
    pos = {}
    consensus = {}
    
    for i in range(It):
        (s, bestScore) = GreedyProfileMotifSearch(sequences, k)
        pos[bestScore]=s
        consensus[bestScore]=extractSeqs(s, sequences, k)
        
    return pos, consensus


s, consensus = GreedyProfileMotifSearchIte(adn, k, 50)

print (s)
print("\nClé : Score de la matrice de fréquence / Valeur : Motifs les plus probables  associés à ce score  :\n")
print (consensus)

{13.0: [0, 3, 2], 11.0: [0, 4, 5], 8.0: [0, 2, 5], 14.0: [5, 4, 4], 10.0: [0, 4, 4], 12.0: [3, 1, 4], 9.0: [0, 0, 0]}

Clé : Score de la matrice de fréquence / Valeur : Motifs les plus probables  associés à ce score  :

{13.0: ['TTGAC', 'CTCAC', 'CACAC'], 11.0: ['TTGAC', 'TCACA', 'ACAAG'], 8.0: ['TTGAC', 'ACTCA', 'ACAAG'], 14.0: ['CCACA', 'TCACA', 'CACAA'], 10.0: ['TTGAC', 'TCACA', 'CACAA'], 12.0: ['ACCCA', 'GACTC', 'CACAA'], 9.0: ['TTGAC', 'AGACT', 'TTCAC']}


In [137]:
#Affiche les sequence consensus de top motifs
def printTopFMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
        print (consensus[motifsSort[i]])

printTopFMotifsScore(consensus, 2)


['CCACA', 'TCACA', 'CACAA']
['TTGAC', 'CTCAC', 'CACAC']


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [138]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [191]:
k=8
n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_1.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
#print (sequencesChip[8], t, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequencesChip = sequencesChip + revSequences
#print(sequencesChip)
s, consensus = GreedyProfileMotifSearchIte(sequencesChip, k, 20)
printTopFMotifsScore(consensus, 10)


['AGAGCCCA', 'ACTGGAGT', 'GAAGACTA', 'GCTCTTCA', 'TGGCTTTG', 'CAGTGTTA', 'ATCAAGTG', 'GCTCGCAG', 'GGAAGGGT', 'CTCCTGCC', 'TAATGATG', 'AAAGGGCA', 'CATGGGGA', 'AGTTGTGT', 'TTCTAGGG', 'TATTTGAC', 'GTATTAGC', 'TTGCATCT', 'ATGAATGC', 'GAAAGAGA', 'AAGCACTG', 'ATCCGTTA', 'AAGATTAC', 'GGGCGTTG', 'ATATCCGA', 'CTAATATC', 'AACCACCA', 'GGTGCAAT', 'GGTGAGCC', 'TTGAGGTG', 'GAACAGAA', 'CAAGTAAG', 'TTTCCTTA', 'CAAGTACA', 'TCTTTCTG', 'AAAAGAAG', 'ACTACTGT', 'GCATGTGT', 'ACACAACC', 'TACCAAAC', 'ACTCACAC', 'CAGCAATT', 'AGGGTGTA', 'TTGTCATT', 'CTTGCAAA', 'CTTCTGAC', 'TAAAAAAT', 'GTATATTG', 'GGTGTGTA', 'GACAGATT', 'AAGCCAAC', 'AAGAAAGA', 'AACCCCAT', 'ATCTTTGT', 'GCACGGCG', 'TTTTTTAT', 'CTAAGAAA', 'GCAGAGGA', 'CGGTAAAC', 'AGTGCGAG', 'TTCCTGGG', 'TCATTTAC', 'GTAACAAT', 'GATGCAAG', 'GACTTCCT', 'GAACACAT', 'ATGTTTTT', 'TTTAATTT', 'ATAAAATT', 'AGTAAAGA', 'TGACCCGA', 'ATTAGTTG', 'TACAGCTT', 'AGTAGCTG', 'ATATATTT', 'GGTGAAAT', 'GCTTATTT', 'TAATAAAA', 'TATACCGA', 'ATATAAAA', 'GCTTAATA', 'TTAAATAA', 'CTAAGTTA', 'AT